## Задание  
Обучить детектор объектов с помощью TensorFlow Object Detection API  
Библиотеки: [Python, Tensorflow]

In [1]:
import os
import urllib.request
import shutil

In [2]:
#Определим каталоги

# Путь к корневой папке
ROOT_DIR = os.path.abspath('.')

# Путь к локальной версии репозитория keras-retinanet
KERAS_RETINANET_PATH = os.path.join(ROOT_DIR, "keras-retinanet")

# Ссылка на скачивание модели
COCO_MODEL_URL = "https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5"
# Путь к предобученной модели
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "resnet50_coco_best_v2.1.0.h5")

## Загрузка и подготовка библиотеки keras-retinanet

In [3]:
if not os.path.exists(KERAS_RETINANET_PATH):
    !git clone https://github.com/fizyr/keras-retinanet.git
    !cd keras-retinanet \
        && git reset --hard abe89380835bc06dff3b97e69fa2b19dd7fd97a8 \
        && pip install . \
        && python setup.py build_ext --inplace

## Загрузка предобученной модели

In [4]:
if not os.path.exists(COCO_MODEL_PATH):
    with urllib.request.urlopen(COCO_MODEL_URL) as resp, open(COCO_MODEL_PATH, 'wb') as out:
        shutil.copyfileobj(resp, out)

## Подготовка данных

Использованы материалы статьи https://danielcorcoranssql.wordpress.com/2019/04/19/keras-retinanet-sea-turtle-training-detection/ и данные репозитория https://github.com/danielc92/oidv4-sea-turtles

In [5]:
import xml.etree.ElementTree as ET
import numpy as np

DATASET_DIR = './content/oidv4-sea-turtles/TRAIN/'
ANNOTATIONS_FILE = 'annotations.csv'
CLASSES_FILE = 'classes.csv'

annotations = []
classes = set([])

for xml_file in [f for f in os.listdir(DATASET_DIR) if f.endswith(".xml")]:
    tree = ET.parse(os.path.join(DATASET_DIR, xml_file))
    root = tree.getroot()
    
    file_name = None
    
    for elem in root:
        if elem.tag == 'filename':
            file_name = os.path.join(DATASET_DIR, elem.text)
            
        if elem.tag == 'object':
            obj_name = None
            coords = []
            
            for subelem in elem:
                if subelem.tag == 'name':
                    obj_name = subelem.text
                if subelem.tag == 'bndbox':
                    for subsubelem in subelem:
                        coords.append(subsubelem.text)
            
            item = [file_name] + coords + [obj_name]
            annotations.append(item)
            classes.add(obj_name)
            
with open(ANNOTATIONS_FILE, 'w') as f:
    for line in annotations:
        f.write('{}\n'.format(",".join(line)))
    
with open(CLASSES_FILE, 'w') as f:
    for i, line in enumerate(classes):
        f.write('{},{}\n'.format(line,i))

## Обучение модели для детектирования объектов

In [6]:
import tensorflow as tf

In [7]:
tf.__version__

'2.9.1'

На текущей версии TF получаем ошибку:

In [8]:
!python keras-retinanet/keras_retinanet/bin/train.py \
    --random-transform \
    --weights "./resnet50_coco_best_v2.1.0.h5" \
    --steps 100 \
    --epochs 20 \
    csv "annotations.csv" "classes.csv"

Creating model, this may take a second...


2022-07-19 18:08:06.578423: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'nvcuda.dll'; dlerror: nvcuda.dll not found
2022-07-19 18:08:06.578454: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-19 18:08:06.581099: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: ITSK-R910T6CC
2022-07-19 18:08:06.581172: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: ITSK-R910T6CC
2022-07-19 18:08:06.582520: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Traceback (most recent call last):
  File "C:\Users\User\Downloads\CV\HW\HW_05\keras-retinanet\keras_reti

К сожалению победить ошибку не удалось, в том числе при использовании %tensorflow_version 1.x в коллабе получаем ошибку:

Using TensorFlow backend.
Creating model, this may take a second...
WARNING:tensorflow:From /tensorflow-1.15.2/python3.7/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
WARNING:tensorflow:From /tensorflow-1.15.2/python3.7/keras/backend/tensorflow_backend.py:4070: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

Traceback (most recent call last):
  File "keras-retinanet/keras_retinanet/bin/train.py", line 530, in <module>
    main()
  File "keras-retinanet/keras_retinanet/bin/train.py", line 490, in main
    config=args.config
  File "keras-retinanet/keras_retinanet/bin/train.py", line 114, in create_models
    model          = model_with_weights(backbone_retinanet(num_classes, num_anchors=num_anchors, modifier=modifier), weights=weights, skip_mismatch=True)
  File "keras-retinanet/keras_retinanet/bin/train.py", line 75, in model_with_weights
    model.load_weights(weights, by_name=True, skip_mismatch=skip_mismatch)
  File "/tensorflow-1.15.2/python3.7/keras/engine/saving.py", line 492, in load_wrapper
    return load_function(*args, **kwargs)
  File "/tensorflow-1.15.2/python3.7/keras/engine/network.py", line 1227, in load_weights
    reshape=reshape)
  File "/tensorflow-1.15.2/python3.7/keras/engine/saving.py", line 1262, in load_weights_from_hdf5_group_by_name
    original_keras_version = f.attrs['keras_version'].decode('utf8')
AttributeError: 'str' object has no attribute 'decode'

## Конвертация обученной модели для инференса

In [9]:
!python keras-retinanet/keras_retinanet/bin/convert_model.py \
    'snapshots/resnet50_csv_20.h5' \
    'snapshots/inference_model.h5'

2022-07-19 18:08:15.686435: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'nvcuda.dll'; dlerror: nvcuda.dll not found
2022-07-19 18:08:15.686465: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-19 18:08:15.689730: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: ITSK-R910T6CC
2022-07-19 18:08:15.689791: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: ITSK-R910T6CC
Traceback (most recent call last):
  File "C:\Users\User\Downloads\CV\HW\HW_05\keras-retinanet\keras_retinanet\bin\convert_model.py", line 97, in <module>
    main()
  File "C:\Users\User\Downloads\CV\HW\HW_05\keras-retinanet\keras_retinanet\bin\convert_model.py", line 75, in main
    model = models.load_model(args.model_in, backbone_name=args.backbone)
  File "C:\Users\User\Downloads\CV\HW\HW_05\keras-retinanet\keras_retinanet\bin\..\..\keras

## Загрузка необходимых библиотек

In [11]:
import time
import imageio
from matplotlib import pyplot as plt
import matplotlib.patches as patches

from keras_retinanet.models import load_model
from keras_retinanet.utils.image import preprocess_image, resize_image
from keras_retinanet.utils.colors import label_color

## Загрузка модели для инференса

In [12]:
model = load_model('snapshots/inference_model.h5', backbone_name='resnet50')

OSError: No file or directory found at snapshots/inference_model.h5

## Загрузка словаря с метками классов

In [13]:
labels_to_names = {}
with open('classes.csv') as f:
    for line in f:
        cls_name, cls_id = line.split(',')
        labels_to_names[int(cls_id.strip())] = cls_name.strip()
print(labels_to_names)

{0: 'Sea turtles'}


## Функция применения модели для детектирования объектов

In [14]:
def detect_objects(image):    
    image_processed = preprocess_image(image[:,:,::-1].copy())
    image_processed, scale = resize_image(image_processed)

    start = time.time()
    boxes, scores, labels = model.predict(image_processed[None, ...])
    print("Processing time: ", time.time() - start)
    boxes /= scale
    return boxes[0], scores[0], labels[0]

## Функция визуализации результатов детектирования объектов

In [15]:
def draw_predictions(image, predictions=None):
    draw = image.copy()
    fig, ax = plt.subplots(1, figsize=(12, 12))
    ax.imshow(draw)

    if predictions is None:
        return
        
    boxes, scores, labels = predictions
    SCORE_THRESHOLD = 0.5
    for box, score, label in zip(boxes, scores, labels):
        if score < SCORE_THRESHOLD:
            break

        box_y = int(box[1])
        box_x = int(box[0])
        box_h = int(box[3]-box[1])
        box_w = int(box[2]-box[0])
        caption = "{} {:.3f}".format(labels_to_names[label], score)
        if 0:
            color = [x/255 for x in label_color(label)]
        else:
            color =  [(0, 1, 0), (1, 1, 0), (1, 0, 1), (1, 0, 0)][label]

        label_size = 20
        plt_scale = float(fig.get_size_inches()[1]) * fig.dpi * draw.shape[0] * label_size / 12545280
        ax.add_patch(patches.Rectangle((box_x, box_y), 
                                 box_w, box_h, 
                                 linewidth=2, edgecolor=color, facecolor='none'))
        ax.add_patch(patches.Rectangle((box_x, box_y-round(26*plt_scale)), 
                                 round(plt_scale*len(caption)*14), round(26*plt_scale), 
                                 linewidth=2, edgecolor=color, facecolor=color))
        ax.text(box_x + round(3*plt_scale), box_y - round(5*plt_scale), caption, fontsize=label_size)

## Детектирование объектов на тестовом изображении

In [16]:
image = imageio.imread('./content/oidv4-sea-turtles/TEST/1ac6dffa6bbf5ac8.jpg')
predictions = detect_objects(image)
draw_predictions(image, predictions)

C:\Users\User\AppData\Local\Temp\ipykernel_9312\320747328.py:1: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread('./content/oidv4-sea-turtles/TEST/1ac6dffa6bbf5ac8.jpg')


NameError: name 'model' is not defined